In [278]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [279]:
data = pd.read_csv('data\movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
75,tt4178092,5000000,58978653,The Gift,Jason Bateman|Rebecca Hall|Joel Edgerton|David...,Joel Edgerton,Not every gift is welcome.,A husband and wife try to reinvigorate their r...,108,Thriller|Mystery,Blumhouse Productions|Blue-Tongue Films|STX En...,7/30/2015,6.6,2015
1057,tt0383010,30000000,54819301,The Three Stooges,Sean Hayes|Will Sasso|Chris Diamantopoulos|Sof...,Bobby Farrelly|Peter Farrelly,Just Say Moe.,While trying to save their childhood orphanage...,92,Comedy,Wessler Entertainment,4/13/2012,4.9,2012
876,tt0295701,70000000,277448382,xXx,Vin Diesel|Asia Argento|Marton Csokas|Samuel L...,Rob Cohen,A New Breed Of Secret Agent.,Xander Cage is your standard adrenaline junkie...,124,Action,Columbia Pictures|Original Film|Revolution Stu...,8/9/2002,5.7,2002
1359,tt0443536,15000000,51053787,Hoodwinked!,Anne Hathaway|Glenn Close|Jim Belushi|Patrick ...,Cory Edwards|Todd Edwards|Tony Leech,Armed And Dangerously Dumb.,The recipes of candies of the goody shops have...,80,Animation|Comedy|Family,The Weinstein Company|Kanbar Entertainment|Blu...,12/16/2005,6.0,2005
173,tt1086772,40000000,123494610,Blended,Drew Barrymore|Adam Sandler|Wendi McLendon-Cov...,Frank Coraci,"Single Dad, No Clue. Single Mum, Flying Solo.","After a bad blind date, a man and woman find t...",117,Comedy,Happy Madison Productions|Karz Entertainment|W...,5/21/2014,6.7,2014


In [280]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [281]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [282]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

data.release_date = pd.to_datetime(data.release_date)
data['profit'] = data.revenue - data.budget
data['release_month'] = data.release_date.dt.month_name()

In [283]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

In [284]:
# create a data frame where multiple values in genres, directors, company & cast columns are unpacked using pd.explode()
data_exploded = data.copy()

def explode_df_column(df, column):
    """
    Explode the column of the dataframe that has multiple values separated by '|' sign and return the dataframe.
    """
    df[column] = df[column].str.split('|')
    df = df.explode(column)
    return df

data_exploded = explode_df_column(data_exploded, 'genres')
data_exploded = explode_df_column(data_exploded, 'director')
data_exploded = explode_df_column(data_exploded, 'cast')
data_exploded = explode_df_column(data_exploded, 'production_companies')

data_exploded.reset_index(drop=True, inplace=True)

# 1. У какого фильма из списка самый большой бюджет?

In [285]:
# тут пишем ваш код для решения данного вопроса:
title, imdb_id = data[data.budget == data.budget.max()][['original_title', 'imdb_id']].values[0]

ВАРИАНТ 2

In [286]:
# можно добавлять разные варианты решения
assert title, imdb_id == data.iloc[data.budget.idxmax()][['original_title', 'imdb_id']]

ФИНАЛЬНЫЙ ОТВЕТ

In [287]:
answers['1'] = f"{title} ({imdb_id})"
# correct

# 2. Какой из фильмов самый длительный (в минутах)?

In [288]:
title, imdb_id = data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']].values[0]

ВАРИАНТ 2

In [289]:
assert title, imdb_id == data.sort_values('runtime', ascending=False)[['original_title', 'imdb_id']].values[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [290]:
answers['2'] = f"{title} ({imdb_id})"
# correct

# 3. Какой из фильмов самый короткий (в минутах)?





In [291]:
title, imdb_id = data[data.runtime == data.runtime.min()][['original_title', 'imdb_id']].values[0]

ВАРИАНТ 2

In [292]:
assert title, imdb_id == data.iloc[data.runtime.idxmin()][['original_title', 'imdb_id']]

ФИНАЛЬНЫЙ ОТВЕТ

In [293]:
answers['3'] = f"{title} ({imdb_id})"
# correct

# 4. Какова средняя длительность фильмов?


In [294]:
mean_runtime = data.runtime.mean()

ВАРИАНТ 2

In [295]:
assert mean_runtime == data.runtime.apply('mean')

ФИНАЛЬНЫЙ ОТВЕТ

In [296]:
answers['4'] = round(mean_runtime, 0)
# correct

# 5. Каково медианное значение длительности фильмов? 

In [297]:
median_runtime = data.runtime.median()

ВАРИАНТ 2

In [298]:
assert median_runtime == data.runtime.apply('median')

ФИНАЛЬНЫЙ ОТВЕТ

In [299]:
answers['5'] = round(median_runtime, 0)
# correct

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [300]:
title, imdb_id = data.iloc[data.profit.idxmax()][['original_title', 'imdb_id']]

ФИНАЛЬНЫЙ ОТВЕТ

In [301]:
answers['6'] = f"{title} ({imdb_id})"
# correct

# 7. Какой фильм самый убыточный? 

In [302]:
title, imdb_id = data.iloc[data.profit.idxmin()][['original_title', 'imdb_id']]

ФИНАЛЬНЫЙ ОТВЕТ

In [303]:
answers['7'] = f"{title} ({imdb_id})"
# correct

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [304]:
n_films = len(data[data.revenue > data.budget])

ВАРИАНТ 2

In [305]:
assert n_films == data.query('revenue > budget').shape[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [306]:
answers['8'] = n_films
# correct

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [307]:
title, imdb_id = data[data.release_year == 2008].sort_values('revenue', ascending=False)[['original_title', 'imdb_id']].values[0]

ВАРИАНТ 2

In [308]:
assert title, imdb_id == data.query('release_year = 2008').sort_values('revenue')[['original_title', 'imdb_id']].values[-1]

ФИНАЛЬНЫЙ ОТВЕТ

In [309]:
answers['9'] = f"{title} ({imdb_id})"
# correct

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [310]:
title, imdb_id = data[(data.release_year >= 2012) & (data.release_year <= 2014)].sort_values('profit')[['original_title', 'imdb_id']].values[0]

ВАРИАНТ 2

In [311]:
assert title, imdb_id == data.query('2012 <= release_year <= 2014').sort_values('profit')[['original_title', 'imdb_id']].values[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [312]:
answers['10'] = f"{title} ({imdb_id})"
# correct

# 11. Какого жанра фильмов больше всего?

In [313]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

c = Counter()
for genre in data.genres:
    for each in genre.split('|'):
        c[each] += 1
genre = [genre for genre, count in c.items() if count == max(c.values())][0]

ВАРИАНТ 2

In [314]:
assert genre == data.genres.str.split('|').explode().value_counts().index[0]

ВАРИАНТ 3

In [315]:
# using exploaded df
assert genre == data_exploded.genres.value_counts().index[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [316]:
answers['11'] = genre
# correct

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [317]:
genre = data_exploded[data_exploded.profit > 0].genres.value_counts().index[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [318]:
answers['12'] = genre
# incorrect

Understood the mistake, the correct code was:

In [319]:
q12 = data[data.profit > 0]
q12.genres.str.split('|').explode().value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [379]:
q13 = data.copy()
q13 = explode_df_column(q13, 'director')
name = q13.groupby('director').revenue.sum().sort_values(ascending=False).index[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [380]:
answers['13'] = name
# correct

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [381]:
q14 = data.copy()
q14 = explode_df_column(q14, 'director')
q14 = explode_df_column(q14, 'genres')
director = q14.query('genres == "Action"').director.value_counts().index[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [382]:
answers['14'] = director
# correct

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [324]:
films_2012 = data_exploded.query('release_year == 2012')
films_2012[films_2012.revenue == films_2012.revenue.max()].cast.value_counts()

Robert Downey Jr.     3
Scarlett Johansson    3
Mark Ruffalo          3
Chris Hemsworth       3
Chris Evans           3
Name: cast, dtype: int64

ФИНАЛЬНЫЙ ОТВЕТ

In [325]:
answers['15'] = 'Chris Hemsworth'
# correct

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [326]:
high_budget_films = data[data.budget > data.budget.mean()].copy()
high_budget_actor = high_budget_films.cast.str.split('|').explode().value_counts().index[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [327]:
answers['16'] = high_budget_actor
# correct

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [328]:
nic_cage_genre = data_exploded.query('cast == "Nicolas Cage"').genres.value_counts().index[0]

ФИНАЛЬНЫЙ ОТВЕТ

In [329]:
answers['17'] = nic_cage_genre
# correct

# 18. Самый убыточный фильм от Paramount Pictures

In [330]:
paramount_films = data[data.production_companies.str.contains('Paramount Pictures')].copy()
title, imdb_id = paramount_films.sort_values('profit')[['original_title', 'imdb_id']].values[0]

In [331]:
answers['18'] = f"{title} ({imdb_id})"
# correct

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [332]:
max_revenue_year = data.groupby('release_year').revenue.sum().sort_values(ascending=False).index[0]

In [333]:
answers['19'] = max_revenue_year
# correct

# 20. Какой самый прибыльный год для студии Warner Bros?

In [334]:
warner_bros_films = data[data.production_companies.str.contains('Warner Bros')].copy()
max_profit_year = warner_bros_films.groupby('release_year').profit.sum().sort_values(ascending=False).index[0]

In [335]:
answers['20'] = max_profit_year
# correct

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [336]:
max_films_month = data.groupby('release_month').original_title.count().sort_values(ascending=False).index[0]

In [337]:
answers['21'] = max_films_month
# correct

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [338]:
max_films_month = data.groupby('release_month').original_title.count().sort_values(ascending=False)
summer_films_num = max_films_month[max_films_month.index.isin(['June', 'July', 'August'])].sum()

In [339]:
answers['22'] = summer_films_num
# correct

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [340]:
winter_films = data_exploded[data_exploded.release_month.isin(['December', 'January', 'February'])].copy()
winter_productive_director = winter_films.groupby('director').original_title.count().sort_values(ascending=False).index[0]

In [341]:
answers['23'] = winter_productive_director
# correct

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [342]:
q24 = data_exploded.copy()
q24['title_len'] = q24.original_title.apply(len)
longest_title_company = q24.groupby('production_companies').title_len.mean().sort_values(ascending=False).index[0]

In [343]:
answers['24'] = longest_title_company
# correct

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [356]:
q25 = data_exploded.copy()
q25['overview_len'] = q25.overview.apply(lambda x: len(x.split(' ')))
longest_overview_company = q25.groupby('production_companies').overview_len.mean().sort_values(ascending=False).index[0]

In [357]:
answers['25'] = longest_overview_company
# correct

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [346]:
one_percent_best_rated_films = data[data.vote_average > data.vote_average.quantile(0.99)].original_title.values

In [347]:
answers['26'] = ['Inside Out', 'The Dark Knight', '12 Years a Slave']
# correct

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [348]:
c = Counter()
for actors in data.cast:
    actors_list = actors.split('|')
    for each in combinations(actors_list, 2):
        c[each] += 1

In [349]:
c.most_common(3)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7)]

In [350]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
# correct

# Submission

In [358]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110.0,
 '5': 107.0,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Christopher Nolan',
 '14': 'Paul W.S. Anderson',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [352]:
# и убедиться что ни чего не пропустил)
len(answers)

27